In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('read_csv').getOrCreate()

In [2]:
t = spark.read.csv('./train.csv', header=True, inferSchema=True)
t.show()

+-------------------+-----+----+-----+
|               date|store|item|sales|
+-------------------+-----+----+-----+
|2013-01-01 00:00:00|    1|   1|   13|
|2013-01-02 00:00:00|    1|   1|   11|
|2013-01-03 00:00:00|    1|   1|   14|
|2013-01-04 00:00:00|    1|   1|   13|
|2013-01-05 00:00:00|    1|   1|   10|
|2013-01-06 00:00:00|    1|   1|   12|
|2013-01-07 00:00:00|    1|   1|   10|
|2013-01-08 00:00:00|    1|   1|    9|
|2013-01-09 00:00:00|    1|   1|   12|
|2013-01-10 00:00:00|    1|   1|    9|
|2013-01-11 00:00:00|    1|   1|    9|
|2013-01-12 00:00:00|    1|   1|    7|
|2013-01-13 00:00:00|    1|   1|   10|
|2013-01-14 00:00:00|    1|   1|   12|
|2013-01-15 00:00:00|    1|   1|    5|
|2013-01-16 00:00:00|    1|   1|    7|
|2013-01-17 00:00:00|    1|   1|   16|
|2013-01-18 00:00:00|    1|   1|    7|
|2013-01-19 00:00:00|    1|   1|   18|
|2013-01-20 00:00:00|    1|   1|   15|
+-------------------+-----+----+-----+
only showing top 20 rows



In [3]:
# date column에서 주차, 요일 추출
from pyspark.sql.functions import *
t = t.withColumn('year', date_format(col('date'), 'Y'))\
    .withColumn('month', date_format(col('date'), 'M'))\
        .withColumn('week', date_format(col('date'), 'W'))\


In [4]:
t.show()

+-------------------+-----+----+-----+----+-----+----+
|               date|store|item|sales|year|month|week|
+-------------------+-----+----+-----+----+-----+----+
|2013-01-01 00:00:00|    1|   1|   13|2013|    1|   1|
|2013-01-02 00:00:00|    1|   1|   11|2013|    1|   1|
|2013-01-03 00:00:00|    1|   1|   14|2013|    1|   1|
|2013-01-04 00:00:00|    1|   1|   13|2013|    1|   1|
|2013-01-05 00:00:00|    1|   1|   10|2013|    1|   1|
|2013-01-06 00:00:00|    1|   1|   12|2013|    1|   2|
|2013-01-07 00:00:00|    1|   1|   10|2013|    1|   2|
|2013-01-08 00:00:00|    1|   1|    9|2013|    1|   2|
|2013-01-09 00:00:00|    1|   1|   12|2013|    1|   2|
|2013-01-10 00:00:00|    1|   1|    9|2013|    1|   2|
|2013-01-11 00:00:00|    1|   1|    9|2013|    1|   2|
|2013-01-12 00:00:00|    1|   1|    7|2013|    1|   2|
|2013-01-13 00:00:00|    1|   1|   10|2013|    1|   3|
|2013-01-14 00:00:00|    1|   1|   12|2013|    1|   3|
|2013-01-15 00:00:00|    1|   1|    5|2013|    1|   3|
|2013-01-1

In [5]:
t.createOrReplaceTempView('rawdf')

In [6]:
# 주 단위 데이터 생성(SQL Query 활용)
tsql = spark.sql('SELECT MIN(year) as year, MIN(month) as month, MIN(week) as week, MIN(store) as store, MIN(item) as item, SUM(sales) as total_sales FROM rawdf GROUP BY (year, month, week, store, item)')\
    .orderBy(['year','month','week','store','item'])
tsql.show()

+----+-----+----+-----+----+-----------+
|year|month|week|store|item|total_sales|
+----+-----+----+-----+----+-----------+
|2013|    1|   1|    1|   1|         61|
|2013|    1|   1|    1|   2|        151|
|2013|    1|   1|    1|   3|         92|
|2013|    1|   1|    1|   4|         60|
|2013|    1|   1|    1|   5|         42|
|2013|    1|   1|    1|   6|        135|
|2013|    1|   1|    1|   7|        156|
|2013|    1|   1|    1|   8|        213|
|2013|    1|   1|    1|   9|        117|
|2013|    1|   1|    1|  10|        183|
|2013|    1|   1|    1|  11|        211|
|2013|    1|   1|    1|  12|        200|
|2013|    1|   1|    1|  13|        212|
|2013|    1|   1|    1|  14|        146|
|2013|    1|   1|    1|  15|        206|
|2013|    1|   1|    1|  16|         74|
|2013|    1|   1|    1|  17|         81|
|2013|    1|   1|    1|  18|        237|
|2013|    1|   1|    1|  19|         88|
|2013|    1|   1|    1|  20|        122|
+----+-----+----+-----+----+-----------+
only showing top

In [7]:
# 주 단위 데이터 생성(DataFrame 활용)
t.groupBy(['year', 'month', 'week', 'store', 'item']).agg(sum('sales').alias('week_sales')).orderBy(['year','month','week','store','item']).show()

+----+-----+----+-----+----+----------+
|year|month|week|store|item|week_sales|
+----+-----+----+-----+----+----------+
|2013|    1|   1|    1|   1|        61|
|2013|    1|   1|    1|   2|       151|
|2013|    1|   1|    1|   3|        92|
|2013|    1|   1|    1|   4|        60|
|2013|    1|   1|    1|   5|        42|
|2013|    1|   1|    1|   6|       135|
|2013|    1|   1|    1|   7|       156|
|2013|    1|   1|    1|   8|       213|
|2013|    1|   1|    1|   9|       117|
|2013|    1|   1|    1|  10|       183|
|2013|    1|   1|    1|  11|       211|
|2013|    1|   1|    1|  12|       200|
|2013|    1|   1|    1|  13|       212|
|2013|    1|   1|    1|  14|       146|
|2013|    1|   1|    1|  15|       206|
|2013|    1|   1|    1|  16|        74|
|2013|    1|   1|    1|  17|        81|
|2013|    1|   1|    1|  18|       237|
|2013|    1|   1|    1|  19|        88|
|2013|    1|   1|    1|  20|       122|
+----+-----+----+-----+----+----------+
only showing top 20 rows



In [8]:
tsql.orderBy(['year','month','week','store','item']).show()

+----+-----+----+-----+----+-----------+
|year|month|week|store|item|total_sales|
+----+-----+----+-----+----+-----------+
|2013|    1|   1|    1|   1|         61|
|2013|    1|   1|    1|   2|        151|
|2013|    1|   1|    1|   3|         92|
|2013|    1|   1|    1|   4|         60|
|2013|    1|   1|    1|   5|         42|
|2013|    1|   1|    1|   6|        135|
|2013|    1|   1|    1|   7|        156|
|2013|    1|   1|    1|   8|        213|
|2013|    1|   1|    1|   9|        117|
|2013|    1|   1|    1|  10|        183|
|2013|    1|   1|    1|  11|        211|
|2013|    1|   1|    1|  12|        200|
|2013|    1|   1|    1|  13|        212|
|2013|    1|   1|    1|  14|        146|
|2013|    1|   1|    1|  15|        206|
|2013|    1|   1|    1|  16|         74|
|2013|    1|   1|    1|  17|         81|
|2013|    1|   1|    1|  18|        237|
|2013|    1|   1|    1|  19|         88|
|2013|    1|   1|    1|  20|        122|
+----+-----+----+-----+----+-----------+
only showing top

In [9]:
t.orderBy(['year','month','week','store','item','date']).show()


+-------------------+-----+----+-----+----+-----+----+
|               date|store|item|sales|year|month|week|
+-------------------+-----+----+-----+----+-----+----+
|2013-01-01 00:00:00|    1|   1|   13|2013|    1|   1|
|2013-01-02 00:00:00|    1|   1|   11|2013|    1|   1|
|2013-01-03 00:00:00|    1|   1|   14|2013|    1|   1|
|2013-01-04 00:00:00|    1|   1|   13|2013|    1|   1|
|2013-01-05 00:00:00|    1|   1|   10|2013|    1|   1|
|2013-01-01 00:00:00|    1|   2|   33|2013|    1|   1|
|2013-01-02 00:00:00|    1|   2|   43|2013|    1|   1|
|2013-01-03 00:00:00|    1|   2|   23|2013|    1|   1|
|2013-01-04 00:00:00|    1|   2|   18|2013|    1|   1|
|2013-01-05 00:00:00|    1|   2|   34|2013|    1|   1|
|2013-01-01 00:00:00|    1|   3|   15|2013|    1|   1|
|2013-01-02 00:00:00|    1|   3|   30|2013|    1|   1|
|2013-01-03 00:00:00|    1|   3|   14|2013|    1|   1|
|2013-01-04 00:00:00|    1|   3|   10|2013|    1|   1|
|2013-01-05 00:00:00|    1|   3|   23|2013|    1|   1|
|2013-01-0